In [1]:
pip install OWSLib pillow

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------- -------------------- 1.8/3.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 9.9 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [lxml]
   ---------------------------------------- 0/2 [lxml]
   ---------------------------------------- 0/2 [lxml]
   -------------------- ------------------- 1/2 [OWSLib]
   -------------------- ------------------- 1/2 [OWSLib]
   -------------------- ------------------- 1/2 [OWSLib]
   -------------------- ------------------- 1/2 [OWSLib]
   -------------------- ------------------- 1/2 [OWSLib]
   -------------------- ------------------- 1/2 [OWSLib]
   -------------------- ------------------- 1/2 [OWSLib]
   ---------------------------------------- 2/2 [OWSLib]

Note: you may need to restart the kernel to use updated packages.


In [1]:
from owslib.wms import WebMapService
wms = WebMapService('https://ows.emodnet-bathymetry.eu/wms', version='1.3.0')
print(list(wms.contents.keys()))


['coastlines', 'emodnet:contours', 'emodnet:download_tiles', 'emodnet:hr_bathymetry_area', 'emodnet:mean', 'emodnet:mean_2016', 'emodnet:mean_2018', 'emodnet:mean_2020', 'emodnet:mean_2022', 'emodnet:mean_atlas_land', 'emodnet:mean_multicolour', 'emodnet:mean_rainbowcolour', 'emodnet:quality_index', 'world:sea_names', 'emodnet:source_references', 'gebco:undersea_features']


In [2]:
from owslib.wfs import WebFeatureService
import geopandas as gpd

# Parámetros
lat, lon = 40.35, 0.40
buffer_deg = 0.2

# Orden correcto de bbox para WFS: lon_min, lat_min, lon_max, lat_max
minx, miny = lon - buffer_deg, lat - buffer_deg
maxx, maxy = lon + buffer_deg, lat + buffer_deg

# Conexión al WFS
wfs = WebFeatureService(
    url='https://ows.emodnet-bathymetry.eu/wfs',
    version='2.0.0'
)

# Petición sin filtros y pidiendo todas las propiedades
response = wfs.getfeature(
    typename='emodnet:contours',
    bbox=(minx, miny, maxx, maxy),
    outputFormat='application/json',
    # si hiciera falta forzar propiedades:
    # propertyname=['elevation']
)

# Cargamos en GeoPandas
gdf = gpd.read_file(response)

# Inspecciona columnas y primeras filas
print("Columnas disponibles:", gdf.columns.tolist())
print(gdf.head())


Columnas disponibles: ['id', 'elevation', 'geometry']
               id  elevation  \
0  contours.23776         50   

                                            geometry  
0  LINESTRING (0.8006 40.4937, 0.8006 40.4936, 0....  


In [3]:
from owslib.wfs import WebFeatureService
import geopandas as gpd
from io import BytesIO

# Misma ventana que antes
lat, lon = 40.35, 0.40
delta = 0.1
min_lon, min_lat = lon - delta, lat - delta
max_lon, max_lat = lon + delta, lat + delta

# Conecta al WFS EMODnet
wfs = WebFeatureService('https://ows.emodnet-bathymetry.eu/wfs', version='2.0.0')

# Descarga la capa de contornos como GeoJSON
resp = wfs.getfeature(
    typename='emodnet:contours',             # <- parámetro singular
    bbox=(min_lon, min_lat, max_lon, max_lat),
    outputFormat='application/json'
)

# Cárgalo en GeoPandas
gdf = gpd.read_file(BytesIO(resp.read()))
gdf.head()  # GeoDataFrame con las líneas de contorno


,id,elevation,geometry
0,contours.23776,50,"LINESTRING (0.8006 40.4937, 0.8006 40.4936, 0...."


In [4]:
import folium

# Parámetros del mapa
lat, lon, zoom = 40.35, 0.40, 10
m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles='OpenStreetMap')

# 1) Overlay: símbolos náuticos
folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

# 2) Overlay: contornos de profundidad (WMS EMODnet)
folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',            # <- usar attr, no attribution
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=1.0
).add_to(m)

# 3) Control de capas
folium.LayerControl().add_to(m)

# 4) Mostrar
m


In [6]:
from ipyleaflet import Map, WMSLayer, LayersControl

m = Map(center=(40, -4), zoom=6)

m.add_layer(WMSLayer(
    url='https://www.ign.es/wms-inspire/pnoa-ma',
    layers='OI.OrthoimageCoverage',
    format='image/jpeg'
))

m.add_layer(WMSLayer(
    url='https://ideihm.covam.es/encwms',
    layers='ENC',
    format='image/png',
    transparent=True
))

m.add_control(LayersControl(position='topright'))
m  # se muestra automáticamente


Map(center=[40, -4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…